In [4]:
import numpy as np
import torch
import esm
import re
from transformers import BertForMaskedLM, BertTokenizer, pipeline

In [5]:
model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()

In [6]:
sequence = "MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKPLSVSYDQATSLRILNNGHAFNVEFDDSQDKAVLKGGPLDGTYRLIQFHFHWGSLDGQGSEHTVDKKKYAAELHLVHWNTKYGDFGKAVQQPDGLAVLGIFLKVGSAKPGLQKVVDVLDSIKTKGKSADFTNFDPRGLLPESLDYWTYPGSLTTPPLLECVTWIVLKEPISVSSEQVLKFRKLNFNGEGEPEELMVDNWRPAQPLKNRQIKASFK"


In [9]:
def get_protbert_embedding(seq):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    tokenizer = BertTokenizer.from_pretrained('Rostlab/prot_bert_bfd', do_lower_case=False )
    model = BertForMaskedLM.from_pretrained("Rostlab/prot_bert_bfd").to(device)
    seq1 = " ".join(list(re.sub(r"[UZOB]", "X", seq)))

    ids = tokenizer(seq1, return_tensors='pt')
    input_ids = torch.tensor(ids['input_ids']).to(device)
    attention_mask = torch.tensor(ids['attention_mask']).to(device)

    with torch.no_grad():
        output = model(input_ids=input_ids, attention_mask=attention_mask)

    output1 = output[0][0][1: -1]
    assert len(seq) == len(output1)

    return output1.cpu()


In [11]:
emb = get_protbert_embedding(sequence)

Some weights of the model checkpoint at Rostlab/prot_bert_bfd were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/var/folders/7s/g4dz7t0n7mq_r8hp256vj64c0000gq/T/ipykernel_30802/4117202380.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(ids[

In [14]:
emb.numpy()

array([[-23.859938, -23.815985, -22.31545 , ..., -23.226543, -24.34729 ,
        -24.115314],
       [-22.448683, -21.592663, -19.956684, ..., -20.410755, -23.648851,
        -21.061636],
       [-24.209427, -22.984352, -23.723406, ..., -24.250395, -22.882277,
        -22.849083],
       ...,
       [-22.058775, -23.30712 , -20.929007, ..., -20.387453, -23.054655,
        -22.220621],
       [-21.16265 , -22.80935 , -21.44013 , ..., -20.688042, -24.13068 ,
        -22.769648],
       [-22.84188 , -23.127182, -23.31618 , ..., -22.15452 , -22.406187,
        -22.116007]], shape=(260, 30), dtype=float32)